In [1]:
import re
import numpy as np
import math
index = [12,13,14,22,24,34,46,116,134,220,223,226,273,276,313,315]

In [2]:
file_name = 'kinet_cond2.inp'
with open(file_name,'r') as f:
    lines = f.readlines()
    f.close()
print(lines)

['ELEMENTS\n', 'e C H\n', 'END\n', '\n', 'SPECIES\n', 'e C CH CH^+ CH2 CH2^+ CH3 CH3^+ CH4 CH4(v13) CH4(v24) CH4^+ C2H2 C2H2(v13) C2H2(v2) C2H2(v5) C2H2^+ C2H3 C2H3^+ C2H4 C2H4(v1) C2H4(v2) C2H4^+ C2H5 C2H5^+\n', 'C2H6 C2H6(v13) C2H6(v24) C2H6^+ C3H5 C3H6 C3H6(v) C3H6^+ C3H7 C3H7^+ C3H8 C3H8(v1) C3H8(v2) C3H8^+ C4H9 C4H9H C2H C5H12 H H2 H2^+ H3^+ H^+\n', 'END\n', '\n', 'BOLSIG\n', 'CH CH2 CH3 CH4 CH4(v13) CH4(v24) C2H2 C2H2(v13) C2H2(v2) C2H2(v5) C2H3 C2H4 C2H4(v1) C2H4(v2) C2H5 C2H6 C2H6(v13) C2H6(v24) C3H6 C3H6(v) C3H7 C3H8 C3H8(v1) C3H8(v2)\n', 'END\n', '\n', 'REACTIONS\n', '# Gas Constant\n', '$ double precision, parameter :: R = 8.314d-3\n', '\n', '# Compensation Parameters\n', '$ double precision, parameter :: f0 = 1.0000d0\n', '$ double precision, parameter :: f1 = 1.0000d0\n', '$ double precision, parameter :: f2 = 1.0000d0\n', '$ double precision, parameter :: f3 = 1.0000d0\n', '$ double precision, parameter :: f4 = 1.0000d0\n', '$ double precision, parameter :: f5 = 1.0000d0\

In [3]:
num_fit = []
for i in index:
    for j in range(len(lines)):
        line = lines[j]
        if f'$ double precision, parameter :: f{i} =' in line:
            match = re.search(r'\b\d+\.\d+d(-?\d{1,2}|0)\b',line)
            if match:
                result = match.group()
                re1 = result.split('d')
                re2 = round(float(re1[0]) * 10**float(re1[1]),5)
                num_fit.append(re2)
fit = np.array(num_fit)

In [4]:
num_ori = []
for i in index:
    for j in range(len(lines)):
        line = lines[j]
        if f'!' in line:
            line2 = line.split('!')
            if line2[1].strip().startswith('f'):
                pass
            else:
                line3 = line2[1].split('*')
                if line3[1].strip() == f'f{i}':
                    result = line3[0].strip()
                    re1 = result.split('d')
                    re2 = round(float(re1[0]) * 10**float(re1[1]),5)
                    num_ori.append(re2)
                    break
ori = np.array(num_ori)

In [5]:
new = fit * ori
new2 = np.round(new,5)
new3 = []
for i in range(len(new2)):
    exponent = int(math.floor(math.log10(abs(new2[i]))))
    mantissa = round(new2[i] / (10**exponent),5)
    expr = f'{mantissa}d{exponent}'
    new3.append(expr)

In [6]:
input_file = 'kinet_blank.inp'
output_file = 'kinet.inp'

with open(input_file, 'r') as f:
    lines = f.readlines()
    f.close()

In [7]:
for i in range(len(index)):
    update_in = []
    for line in lines:
        if f'! f{index[i]} *' in line:
            line2 = line.replace(f'! f{index[i]}',f'! {new3[i]} * f{index[i]} ')
            update_in.append(line2)
        else:
            update_in.append(line)
    lines = update_in

In [8]:
with open(output_file, 'w') as file:
    file.writelines(lines)